In [1]:
import numpy as np
import asyncio
#from mavsdk import System
import mediapipe as mp
import PIL
from PIL import ImageDraw

In [2]:
#drone =  System(mavsdk_server_address='localhost', port=50051)

model = mp.solutions.face_detection.FaceDetection()

In [21]:
async def setup():
    await drone.connect(system_address="udp://:14540")
    
    print("Waiting for drone to connect...")
    async for state in drone.core.connection_state():
        if state.is_connected:
            print(f"-- Connected to drone!")
            break
    
    print("Waiting for drone to have a global position estimate...")
    async for health in drone.telemetry.health():
        if health.is_global_position_ok and health.is_home_position_ok:
            print("-- Global position estimate OK")
            break
    
    print("-- Arming")
    await drone.action.arm()
    
    print("-- Taking off")
    await drone.action.takeoff()

    await asyncio.sleep(5)
    
    print("-- Starting manual control")
    await drone.manual_control.start_position_control()
    print('Drone Ready!')
    

def move_vert(d):
    #await drone.manual_control.set_manual_control_input(0.0, 0.0, d, 0.)
    m = ''
    if d > 0:
        m = 'move up'
    elif d<0:
        m = 'move down'
    print(m)
    
def move_hori(d):
    #await drone.manual_control.set_manual_control_input(0.0, 0.0, 0.0, d)
    m = ''
    if d > 0:
        m = 'move right'
    elif d<0:
        m = 'move left'
    print(m)

# t
def _capture():
    return np.array(PIL.Image.open('face/face.jpg'), 'uint8')

def main():
    img_array = _capture()
    d1, d2 = predict(img_array)
    move_hori(d1)
    move_vert(d2)
    


In [22]:
def parse_pb2(pb2):
    s = pb2.score[0]
    x = pb2.location_data.relative_bounding_box.xmin
    y = pb2.location_data.relative_bounding_box.ymin
    w = pb2.location_data.relative_bounding_box.width
    h = pb2.location_data.relative_bounding_box.height
    return (s, x, y, w, h)

def detect(img_path_or_array):
    if isinstance(img_path_or_array, str):
        img_array = PIL.Image.open(file_path_or_array)
        img_array = np.array(img, dtype='uint8')
    else:
        img_array = img_path_or_array
    
    r = model.process(img_array)
    if r.detections is None:
        return None
    r = np.array([parse_pb2(i) for i in r.detections])
    best = r[np.argmax(r[...,0], axis=-1)]
    h = img_array.shape[-3]
    w = img_array.shape[-2]
    best_score = best[0]
    best_bbox = [w*best[1], h*best[2], w*best[3], h*best[4]]
    return best_score, best_bbox


def draw_bbox_on_image(img, bbox):
    draw = ImageDraw.Draw(img)
    draw.rectangle([(bbox[0],bbox[1]),(bbox[0]+bbox[2],bbox[1]+bbox[3])], outline=(255,32,142), width=2)

def detect_and_draw(filepath):
    img = PIL.Image.open(filepath)
    img_array = np.array(img, 'uint8')
    try:
        score, bbox = detect(img_array)
        draw_bbox_on_image(img, bbox)
        img.show()
    except Exception as e:
        print('error:', e)
    

In [85]:
def predict(img_array):
    r = detect(img_array)
    if r is None:
        return
    s, b = r
    img_size = img_array.shape[:2]
    d1, d2 = get_direction_to_move(b, img_size)
    return d1, d2
    

def get_direction_to_move(coords, img_size, th=0.2):
    '''calcula quanto devera ser descolado vertical e horizontalmente'''
    IMG_SIZE = img_size
    if len(coords) != 4:
        raise ValueError(f'invalid coordinates size {len(coords)}, expected 4 (x1,y1,x2,y2)')
    
    xmin, ymin, width, height = coords
    
    th_h = height*th # th*IMG_SIZE[0]
    th_w = width*th # th*IMG_SIZE[1]
    
    x_center = xmin + width/2
    y_center = ymin + height/2
    
    W_CENTER = IMG_SIZE[1]/2
    H_CENTER = IMG_SIZE[0]/2
    
    D1 = 0
    D2 = 0
    
    if x_center > W_CENTER and  x_center-W_CENTER >= th_w:
        D1 += 0.5
        
    elif x_center < W_CENTER and W_CENTER-x_center >= th_w:
        D1 -= 0.5
    
    if y_center > H_CENTER and y_center-H_CENTER >= th_h:
        D2 -= 0.5
    
    elif y_center < H_CENTER and H_CENTER-y_center >= th_h:
        D2 += 0.5
    return D1, D2
        



In [86]:
main()

In [68]:
14.280988848209383/0.05

285.6197769641876